In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [13]:

csv = ".\csv\PlayerStat.csv"

df = pd.read_csv(csv)
df.isnull().sum()
df.fillna(0, inplace=True)  # or use df.dropna()
df_time_range = df[df['season']>2010]
df_time_range = df_time_range[['player_id','player','fg_per_game', 'x3p_percent', 'x2p_per_game',
       'ft_per_game', 
       'trb_per_game', 'ast_per_game',
       'stl_per_game', 'blk_per_game', 'tov_per_game', 
       'pts_per_game']]
df_time_range

,player_id,player,fg_per_game,x3p_percent,x2p_per_game,ft_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pts_per_game
0,5025,A.J. Green,2.6,0.442,0.3,0.3,2.3,1.3,0.5,0.1,0.6,7.7
1,5026,A.J. Lawson,1.0,0.400,0.3,0.3,1.0,0.0,0.0,0.0,0.0,3.0
2,5210,AJ Johnson,0.6,1.000,0.4,0.0,0.4,0.2,0.0,0.0,0.4,1.4
3,4219,Aaron Gordon,4.8,0.440,3.5,2.9,5.8,3.1,0.5,0.2,1.4,13.7
4,4582,Aaron Holiday,1.3,0.425,0.5,0.5,0.9,1.2,0.2,0.1,0.3,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...
9654,3981,Zabian Dowdell,2.1,0.300,2.0,0.7,0.8,2.1,0.8,0.1,1.2,5.0
9655,3363,Zach Randolph,8.0,0.186,7.9,4.0,12.2,2.2,0.8,0.3,2.0,20.1
9656,3494,Zaza Pachulia,1.4,0.000,1.4,1.7,4.2,0.7,0.4,0.3,0.9,4.4
9657,3100,Zydrunas Ilgauskas,2.3,0.000,2.3,0.5,4.0,0.4,0.3,0.8,0.7,5.0


In [21]:
## standardization
scaler = StandardScaler()
scaled_features = scaler.fit_transform([df_time_range['pts_per_game','ast_per_game','trb_per_game']])
scaled_features

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 9659))